# MCMC CALIBRATION TECHNICS IN CONTEXT OF  INFECTIOUS DISEASE MODELING

## Prerequies

In [ ]:
# pip install summerepi2 estival jax==0.4.24 jaxlib==0.4.24 nevergrad jupyter pymc numpyro

In [ ]:
# pip install multiprocess
!pip install --upgrade --force-reinstall multiprocess


In [1]:
# This is required for pymc parallel evaluation in notebooks

import multiprocess as mp
import platform

if platform.system() != "Windows":
    
    mp.set_start_method('forkserver')

In [4]:
import pandas as pd
from scipy import stats
import numpy as np
import plotly.express as px
pd.options.plotting.backend = "plotly"
import summer2
from summer2 import CompartmentalModel # type: ignore
from summer2.parameters import Parameter


# The following imports are the 'building blocks' of estival models

# Targets represent data we are trying to fit to
from estival import targets as est
import estival

# We specify parameters using (Bayesian) priors
from estival import priors as esp

# Finally we combine these with our summer2 model in a BayesianCompartmentalModel (BCM)
import estival.model
from estival.model import BayesianCompartmentalModel

# Importing the pymc package

import pymc as pm
from estival.wrappers import pymc as epm

In [5]:
esp.GammaPrior?

Init signature: esp.GammaPrior(name: str, shape: float, scale: float, size: int = 1)
Docstring:      A gamma distributed prior
File:           c:\users\abdou\anaconda3\envs\emulearn\lib\site-packages\estival\priors.py
Type:           ABCMeta
Subclasses:     

## Calibration Data

In [53]:
#An example of data for the calibration 

output_labels = {"index": "time", "value": "number infectious"}

data = pd.DataFrame(
    {"active_cases":
     {
         60.0: 3000.0,
         80.0: 8500.0,
         100.0: 21000.0,
         120.0: 40000.0,
         140.0: 44000.0,
         160.0: 30000.0,
         180.0: 16000.0,
         200.0: 7000.0,
     },
    },
)

targets_data = data["active_cases"]
targets_data
data["active_cases"].plot(kind="scatter", labels=output_labels)

In [47]:
#Specify a Truncated normal target with a free dispersion parameter
targets = [
    est.TruncatedNormalTarget("active_cases", targets_data, (0.0,np.inf),
        #esp.UniformPrior("incidence_dispersion",(0.1, obs_noisy.max()*0.1))) #Calibration de l'ecart type
        0.5) #Utiliser un nombre fixe pour ne pas l'inclure dans la calibration
]     


# Uniform priors over our 2 model parameters
priors = [
    esp.UniformPrior("contact_rate", (0.0,0.5)),
    esp.GammaPrior("recovery_rate", 2.0, 0.1),# (0.01,1.0)),
]


In [ ]:
est.TruncatedNormalTarget?

## Model Definition and Configuration

A mechanistic model (ODE-Based) model discribing Infectious Disease transmission.

In [11]:
from models.models import Build_model

In [13]:
def get_sir_model(
    config: dict,
) -> CompartmentalModel:

    model = CompartmentalModel(
        times=(0.0, config["end_time"]),
        compartments=(
            "susceptible", 
            "infectious", 
            "recovered",
        ),
        infectious_compartments=("infectious",),
    )
    model.set_initial_population(
        distribution=
        {
            "susceptible": config["population"] - config["seed"], 
            "infectious": config["seed"],
        },
    )
    
    model.add_infection_frequency_flow(
        name="infection", 
        contact_rate=Parameter("contact_rate"), 
        source="susceptible", 
        dest="infectious",
    )
    model.add_transition_flow(
        name="recovery",
        fractional_rate=Parameter("recovery_rate"),
        source="infectious",
        dest="recovered",
    )
    model.request_output_for_compartments(name="active_cases", compartments=["infectious"])

    return model

In [26]:
model_config = {
    "population": 1e6,
    "seed": 100.0,
    "end_time": 365.0,
}
parameters = {
    "contact_rate": 0.3,
    "recovery_rate": 0.1,
}

sir_model = get_sir_model(model_config)




In [ ]:
# #sir_model = get_sir_model(model_config)
# sir_model.request_output_for_compartments(
#         "infectious_population_size", "infection"
#     )

In [48]:


sir_model.run(parameters)


pd.DataFrame(
    {
        "modelled": sir_model.get_outputs_df()["infectious"],
        "observed": data["active_cases"],
    }
).plot(kind="scatter", labels=output_labels)

In [49]:
#Defining  a Bayesian Compartmental Model
bcm = BayesianCompartmentalModel(sir_model, parameters, priors, targets)

In [50]:
initial_parameters = {
    "contact_rate": 0.15,
    "recovery_rate": 0.1,
}
bcm.run(initial_parameters)

ResultsData(derived_outputs=       active_cases
0.0      100.000000
1.0      105.125413
2.0      110.513262
3.0      116.176959
4.0      122.130595
...             ...
361.0    113.220878
362.0    109.063468
363.0    105.058609
364.0    101.200710
365.0     97.484386

[366 rows x 1 columns], extras={'ll_components': {'active_cases': Array(-8.77665369e+08, dtype=float64)}, 'loglikelihood': Array(-8.77665369e+08, dtype=float64), 'logprior': 1.9957322735539909, 'logposterior': Array(-8.77665367e+08, dtype=float64)})

In [51]:
with pm.Model() as model:
    
    # This is all you need - a single call to use_model
    variables = epm.use_model(bcm)
    #variables_defp?
    #print(model.initial_point())

    # The log-posterior value can also be output, but may incur additional overhead
    # Use jacobian=False to get the unwarped value (ie just the 'native' density of the priors
    # without transformation correction factors)
    # pm.Deterministic("logp", model.logp(jacobian=False))
    
    # Now call a sampler using the variables from use_model
    # In this case we use the Differential Evolution Metropolis sampler
    # See the PyMC docs for more details
    idata_DEM = pm.sample(step=[pm.DEMetropolisZ(variables, S = 0.5,proposal_dist = pm.NormalProposal)], initvals = initial_parameters, draws=1000,tune = 1000 ,cores=4,chains=4)
    #print(model.initial_point()["contact_rate_interval__"])

Multiprocess sampling (4 chains in 4 jobs)
DEMetropolisZ: [contact_rate, recovery_rate]


Sampling 4 chains for 1_000 tune and 1_000 draw iterations (4_000 + 4_000 draws total) took 50 seconds.
The rhat statistic is larger than 1.01 for some parameters. This indicates problems during sampling. See https://arxiv.org/abs/1903.08008 for details
The effective sample size per chain is smaller than 100 for some parameters.  A higher number is needed for reliable rhat and ess computation. See https://arxiv.org/abs/1903.08008 for details
